# Brain Stroke Diagnosos Prediction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

In [2]:
# importing the data
data = pd.read_csv("../datasets/train.csv")
test_data = pd.read_csv("../datasets/test.csv")

In [3]:
print(data.shape, test_data.shape)

(15304, 12) (10204, 11)


In [4]:
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [5]:
data["stroke"].value_counts()

stroke
0    14672
1      632
Name: count, dtype: int64

In [6]:
# missing values
data.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15304 entries, 0 to 15303
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 15304 non-null  int64  
 1   gender             15304 non-null  object 
 2   age                15304 non-null  float64
 3   hypertension       15304 non-null  int64  
 4   heart_disease      15304 non-null  int64  
 5   ever_married       15304 non-null  object 
 6   work_type          15304 non-null  object 
 7   Residence_type     15304 non-null  object 
 8   avg_glucose_level  15304 non-null  float64
 9   bmi                15304 non-null  float64
 10  smoking_status     15304 non-null  object 
 11  stroke             15304 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 1.4+ MB


In [8]:
data.sample(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
14604,14604,Female,55.0,0,0,Yes,Private,Urban,88.88,28.4,never smoked,0
525,525,Female,19.0,0,0,No,Private,Rural,100.61,22.0,Unknown,0
13118,13118,Female,27.0,0,0,Yes,Private,Urban,94.19,22.3,never smoked,0
12523,12523,Female,31.0,0,0,Yes,Private,Urban,72.49,41.2,Unknown,0
7268,7268,Female,22.0,0,0,No,Private,Urban,84.49,46.1,never smoked,0
3954,3954,Female,20.0,0,0,Yes,Private,Rural,92.74,20.1,Unknown,0
4045,4045,Male,18.0,0,0,No,Private,Urban,101.09,17.3,formerly smoked,0
9782,9782,Male,45.0,0,0,Yes,Govt_job,Urban,99.47,27.7,never smoked,0
4227,4227,Male,53.0,0,0,Yes,Private,Urban,90.77,31.6,Unknown,0
15083,15083,Male,76.0,0,0,Yes,Self-employed,Urban,72.96,36.0,formerly smoked,0


In [9]:
data = data.drop("id", axis=1)
data.sample(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
7408,Female,15.00,0,0,No,Private,Rural,90.44,18.4,formerly smoked,0
6068,Female,40.00,0,0,No,Private,Urban,93.97,25.7,Unknown,0
5111,Male,82.00,0,0,Yes,Private,Urban,91.25,25.5,formerly smoked,1
3047,Male,17.00,0,0,No,Private,Urban,76.74,29.2,Unknown,0
9558,Female,25.00,0,0,No,Private,Rural,81.54,37.9,Unknown,0
9359,Male,43.00,0,0,Yes,Self-employed,Urban,78.59,43.6,smokes,0
1320,Male,41.00,0,0,Yes,Private,Urban,73.29,37.3,never smoked,0
13469,Female,57.00,0,0,Yes,Private,Rural,114.76,22.1,never smoked,0
5646,Female,12.00,0,0,No,children,Rural,116.04,27.8,Unknown,0
2244,Male,1.16,0,0,No,children,Urban,109.30,20.1,Unknown,0


In [10]:
data["work_type"].value_counts()

work_type
Private          9752
children         2038
Self-employed    1939
Govt_job         1533
Never_worked       42
Name: count, dtype: int64

In [11]:
data["smoking_status"].value_counts()

smoking_status
never smoked       6281
Unknown            4543
formerly smoked    2337
smokes             2143
Name: count, dtype: int64

In [12]:
data["Residence_type"].value_counts()

Residence_type
Rural    7664
Urban    7640
Name: count, dtype: int64

In [13]:
data.sample(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
11636,Male,44.00,0,0,Yes,Private,Rural,86.78,25.5,Unknown,0
7169,Female,1.24,0,0,No,children,Rural,74.70,21.8,Unknown,0
369,Male,59.00,1,0,Yes,Private,Rural,86.21,30.1,formerly smoked,0
9778,Female,58.00,0,0,Yes,Private,Rural,127.40,22.5,formerly smoked,0
14556,Male,43.00,0,0,Yes,Private,Rural,84.04,30.6,Unknown,0
7358,Female,66.00,0,0,Yes,Private,Urban,86.11,28.7,formerly smoked,0
13680,Male,42.00,0,0,Yes,Private,Rural,80.43,45.2,smokes,1
8317,Male,55.00,0,0,Yes,Private,Rural,75.82,28.3,never smoked,0
1350,Male,49.00,0,0,Yes,Private,Urban,100.61,30.8,Unknown,0
13466,Female,41.00,0,0,Yes,Private,Urban,82.91,24.7,Unknown,0


In [14]:
data["age"].unique

<bound method Series.unique of 0        28.0
1        33.0
2        42.0
3        56.0
4        24.0
         ... 
15299    22.0
15300    46.0
15301    75.0
15302    46.0
15303    14.0
Name: age, Length: 15304, dtype: float64>

In [15]:
data["gender"].value_counts()

gender
Female    9446
Male      5857
Other        1
Name: count, dtype: int64

In [16]:
data["avg_glucose_level"].unique

<bound method Series.unique of 0         79.53
1         78.44
2        103.00
3         64.87
4         73.36
          ...  
15299     72.63
15300    101.19
15301     87.69
15302    101.13
15303     85.12
Name: avg_glucose_level, Length: 15304, dtype: float64>

In [17]:
data["bmi"].unique

<bound method Series.unique of 0        31.1
1        23.9
2        40.3
3        28.8
4        28.8
         ... 
15299    19.5
15300    32.1
15301    26.2
15302    22.5
15303    24.7
Name: bmi, Length: 15304, dtype: float64>

In [18]:
data["smoking_status"].value_counts()

smoking_status
never smoked       6281
Unknown            4543
formerly smoked    2337
smokes             2143
Name: count, dtype: int64

In [19]:
data.sample(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
8325,Male,15.0,0,0,No,Private,Rural,84.20,26.5,never smoked,0
2105,Female,12.0,0,0,No,children,Rural,95.08,35.1,Unknown,0
5074,Female,16.0,0,0,No,Private,Rural,106.85,22.9,Unknown,0
2981,Female,64.0,0,0,Yes,Govt_job,Urban,100.31,31.4,Unknown,0
3807,Female,57.0,0,0,Yes,Private,Rural,84.84,21.4,never smoked,0
15063,Female,40.0,0,0,Yes,Private,Rural,72.49,25.9,smokes,0
1908,Male,78.0,1,0,Yes,Govt_job,Rural,216.90,45.0,never smoked,0
10822,Female,2.0,0,0,No,children,Urban,108.80,17.7,Unknown,0
11569,Female,9.0,0,0,No,children,Rural,98.55,21.9,Unknown,0
140,Male,55.0,0,0,Yes,Private,Rural,89.43,27.8,formerly smoked,0


In [20]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,15304.0,41.417708,21.444673,0.08,26.0,43.00,57.00,82.0
hypertension,15304.0,0.049726,0.217384,0.00,0.0,0.00,0.00,1.0
heart_disease,15304.0,0.023327,0.150946,0.00,0.0,0.00,0.00,1.0
avg_glucose_level,15304.0,89.039853,25.476102,55.22,74.9,85.12,96.98,267.6
bmi,15304.0,28.112721,6.722315,10.30,23.5,27.60,32.00,80.1
stroke,15304.0,0.041296,0.198981,0.00,0.0,0.00,0.00,1.0


In [21]:
data.nunique()

gender                  3
age                   106
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3740
bmi                   407
smoking_status          4
stroke                  2
dtype: int64

In [22]:
data.duplicated().sum()

np.int64(0)

In [23]:
data["age"] = np.ceil(data["age"])
test_data["age"] = np.ceil(data["age"])

In [24]:
data.sample(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
4671,Male,27.0,0,0,No,Govt_job,Rural,93.58,35.9,smokes,0
8772,Female,30.0,0,0,Yes,Private,Rural,91.26,38.1,Unknown,0
8091,Male,32.0,1,0,No,Private,Urban,87.23,34.7,never smoked,0
8178,Female,55.0,0,0,Yes,Private,Rural,68.88,27.2,Unknown,0
13269,Male,58.0,0,0,Yes,Govt_job,Urban,69.16,24.9,Unknown,0
4981,Female,40.0,0,0,Yes,Self-employed,Urban,65.59,30.1,never smoked,0
2453,Female,29.0,0,0,Yes,Private,Urban,91.68,24.2,Unknown,0
14076,Female,34.0,0,0,Yes,Govt_job,Urban,61.34,34.3,never smoked,0
1575,Female,17.0,0,0,No,Private,Rural,81.74,24.4,never smoked,0
9840,Female,32.0,0,0,Yes,Private,Urban,93.24,24.8,never smoked,0


In [25]:
test_data.sample(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
59,15363,Male,18.0,0,0,No,Private,Urban,104.51,20.0,never smoked
2777,18081,Female,48.0,0,0,Yes,Private,Rural,73.56,36.1,never smoked
5174,20478,Female,31.0,0,0,No,Private,Rural,96.77,24.3,never smoked
4214,19518,Male,28.0,1,0,Yes,Private,Urban,72.67,28.4,never smoked
8576,23880,Male,63.0,0,0,Yes,Self-employed,Rural,108.64,27.3,never smoked
5572,20876,Female,30.0,0,0,Yes,Private,Urban,112.72,22.4,smokes
5852,21156,Male,32.0,0,0,Yes,Private,Urban,111.27,39.4,never smoked
8629,23933,Female,43.0,0,0,No,Private,Urban,87.15,24.0,never smoked
2161,17465,Male,53.0,0,0,Yes,Self-employed,Rural,77.72,25.9,formerly smoked
4700,20004,Male,14.0,0,0,No,Private,Rural,85.82,35.6,never smoked


In [26]:
test_data = test_data.drop("id", axis=1)
test_data.sample(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
8347,Female,3.0,0,0,Yes,Private,Urban,85.79,27.9,formerly smoked
9568,Female,79.0,0,0,No,Private,Rural,60.22,21.0,smokes
644,Female,34.0,0,0,Yes,Private,Rural,100.65,22.7,Unknown
6951,Female,59.0,0,0,Yes,Govt_job,Urban,72.37,24.7,Unknown
5140,Female,5.0,1,0,Yes,Private,Urban,74.01,31.2,never smoked
7298,Male,24.0,0,0,Yes,Private,Rural,68.37,32.8,never smoked
2858,Male,79.0,0,0,Yes,Govt_job,Rural,222.29,30.1,smokes
8321,Female,59.0,0,0,Yes,Private,Rural,89.43,43.4,never smoked
5259,Female,55.0,0,0,Yes,Private,Rural,93.67,20.6,formerly smoked
5278,Female,43.0,0,0,Yes,Private,Rural,91.58,29.4,formerly smoked


In [27]:
# removing the other gender from training
data.drop(data[data.gender == "Other"].index, inplace=True)
data["gender"].value_counts()

gender
Female    9446
Male      5857
Name: count, dtype: int64

In [28]:
test_data["gender"].value_counts()

gender
Female    6338
Male      3865
Other        1
Name: count, dtype: int64

In [29]:
# removing the other gender from test_data
test_data.drop(test_data[test_data.gender == "Other"].index, inplace=True)
test_data["gender"].value_counts()

gender
Female    6338
Male      3865
Name: count, dtype: int64

## Exploratory Data Analysis

In [41]:
# seaborn and plot default styles
sns.set_style("darkgrid")
plot_color = ["#B7410E", "#008080"]
sns.set_palette(["#B7410E", "black"])

In [31]:
# filtering data for stroke and non stroke
stroke_data = data[data["stroke"] == 1]
non_stroke_data = data[data["stroke"] == 0]

In [32]:
stroke_data.sample(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
13111,Female,53.0,0,0,Yes,Private,Rural,216.88,31.4,smokes,1
10219,Female,53.0,0,0,Yes,Private,Rural,84.88,34.7,Unknown,1
7504,Female,44.0,0,0,Yes,Private,Rural,116.66,24.1,never smoked,1
407,Female,78.0,0,0,Yes,Self-employed,Rural,105.58,25.3,never smoked,1
2169,Male,57.0,0,0,Yes,Private,Urban,111.81,27.9,Unknown,1
864,Female,65.0,0,0,Yes,Govt_job,Rural,180.93,31.9,never smoked,1
14971,Female,79.0,0,0,Yes,Self-employed,Urban,72.54,22.1,never smoked,1
1107,Male,67.0,0,0,Yes,Private,Rural,97.06,30.7,smokes,1
385,Female,79.0,0,0,Yes,Private,Urban,89.28,34.4,never smoked,1
3301,Female,58.0,0,1,Yes,Private,Urban,93.48,33.9,Unknown,1


In [33]:
non_stroke_data.sample(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
9602,Female,57.0,0,0,Yes,Private,Rural,111.81,21.4,formerly smoked,0
4246,Male,66.0,0,0,Yes,Private,Rural,95.19,30.9,formerly smoked,0
3250,Female,47.0,0,0,Yes,Govt_job,Rural,94.67,24.2,never smoked,0
2846,Male,10.0,0,0,No,children,Urban,110.69,28.5,Unknown,0
10552,Female,33.0,0,0,Yes,Self-employed,Urban,84.68,29.5,never smoked,0
13875,Female,44.0,0,0,Yes,Private,Rural,70.48,33.5,never smoked,0
7720,Male,29.0,0,0,Yes,Private,Urban,78.80,28.4,smokes,0
3483,Male,51.0,0,0,Yes,Govt_job,Rural,72.20,28.5,never smoked,0
2224,Female,46.0,0,0,Yes,Private,Rural,78.11,35.3,formerly smoked,0
1882,Female,56.0,0,0,Yes,Private,Urban,80.08,25.6,never smoked,0


In [34]:
stroke_count = data["stroke"].value_counts()
stroke_count

stroke
0    14671
1      632
Name: count, dtype: int64

In [ ]:
# plot for stroke and non stroke: pie chart: univariate analysis
fig = px.pie(
    stroke_count,
    values = stroke_count.values,
    names = stroke_count.index,
    title = "Proportion of Stroke Cases",
    color_discrete_sequence = plot_color
)
fig.update_layout(
    legend_title_text = "Stroke Status",
    title_x = 0.3,
    title_y = 0.95,
    paper_bgcolor="lightgray",
    plot_bgcolor="black",
    width=1000,
    height=600
)
fig.show()

In [68]:
# bivariate analysis
def custom_hist(column:str, titles:list)->go.Figure:
    fig = sp.make_subplots(
        rows=1,
        cols=2,
        subplot_titles = ['Stroke', 'No Stroke']
    )
    stroke_hist = px.histogram(
        stroke_data,
        x = column,
        color_discrete_sequence = ["#B7410E"]
    )
    non_stroke_hist = px.histogram(
        non_stroke_data,
        x = column,
        color_discrete_sequence = ["#008080"]
    )
    fig.add_trace(stroke_hist.data[0], row=1, col=1)
    fig.add_trace(non_stroke_hist.data[0], row=1, col=2)
    fig.update_layout(title = titles[0], title_x = 0.5)
    fig.update_xaxes(title_text = titles[1], row=1, col=1)
    fig.update_xaxes(title_text = titles[1], row=1, col=2)
    fig.update_yaxes(title_text = "Count")

    return fig

In [ ]:
# glucose levels distribution by stroke
column = "avg_glucose_level"
glucose_titles = [
    "Average Glucose Level Distribution by Stroke",
    "Average Glucose Level"
]
glu_fig = custom_hist(column, glucose_titles)
glu_fig.show()

In [70]:
# age distribution by stroke
age_column = "age"
age_titles = [
    "Age Distribution by Stroke",
    "Age"
]
age_fig = custom_hist(age_column, age_titles)
age_fig.show()